In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. 학습 준비

### 1) 라이브러리 다운

In [ ]:
# 싹 정리
!pip uninstall -y torch torchvision torchaudio triton bitsandbytes

# PyTorch 2.1 라인 (cu121)
!pip install --index-url https://download.pytorch.org/whl/cu121 torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2

# Triton: 2.1.0 (bnb가 기대하는 triton.ops 포함)
!pip install triton==2.1.0

# bitsandbytes: 0.43.1
!pip install bitsandbytes==0.43.1

# (이미 있으면 생략 가능) TRL 스택
!pip install "trl==0.8.6" "transformers==4.40.2" "peft==0.10.0" "accelerate==0.30.1"


Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: triton 3.2.0
Uninstalling triton-3.2.0:
  Successfully uninstalled triton-3.2.0
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 614.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 123.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 충돌만 해소
!pip install -U "sentence-transformers==3.0.1"

# (권장) 바로 런타임 재시작
import IPython; IPython.get_ipython().kernel.do_shutdown(True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 5.0.0
    Uninstalling sentence-transformers-5.0.0:
      Successfully uninstalled sentence-transformers-5.0.0


{'status': 'ok', 'restart': True}

In [ ]:
# 1) NumPy 2 제거 후 1.x로 고정 (Colab에선 1.26.4가 가장 무난)
!pip uninstall -y numpy
!pip install "numpy==1.26.4"

# 2) (선택) sentence-transformers 충돌도 같이 정리하려면, A안 사용 중이니 이 버전으로 고정
!pip install -U "sentence-transformers==3.0.1"


Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 111.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.


^C


In [ ]:
import numpy, torch
print("numpy:", numpy.__version__)
print("torch:", torch.__version__)
print("torch.uint64 ok?", hasattr(torch, "uint64"))


numpy: 2.0.2
torch: 2.6.0+cu124
torch.uint64 ok? True


### 2) 데이터 로드/분리 + 증강

In [ ]:
# === 1) 기본 준비 ===
import os, json, re, random, math
import pandas as pd
from sklearn.model_selection import train_test_split

LABELS = ["분노", "기쁨", "슬픔", "불안", "상처", "당황", "중립"]
SEED = 42
random.seed(SEED)

# 파일 경로
# DATA_XLSX = "/content/drive/MyDrive/금융공모전/merged_emotion_dataset.xlsx"

# # 데이터 로드 & 정리
# df = pd.read_excel(DATA_XLSX)
# df = df.dropna(subset=["Sentence","Emotion"])
# df = df[df["Emotion"].isin(LABELS)].reset_index(drop=True)

# # === 2) 분리: 증강 이전에 stratify로 고정 ===
# train_df, temp_df = train_test_split(
#     df, test_size=0.2, random_state=SEED, stratify=df["Emotion"]
# )
# val_df, test_df = train_test_split(
#     temp_df, test_size=0.5, random_state=SEED, stratify=temp_df["Emotion"]
# )

# print("Train dist:\n", train_df["Emotion"].value_counts().sort_index(), "\n")
# print("Val dist:\n",   val_df["Emotion"].value_counts().sort_index(), "\n")
# print("Test dist:\n",  test_df["Emotion"].value_counts().sort_index(), "\n")


In [ ]:
# === 3) GPT 증강 유틸 ===
from openai import OpenAI

# !! 절대 하드코딩하지 말고 환경변수로 설정하세요.
# os.environ["OPENAI_API_KEY"] = "..."
client = OpenAI(api_key="###")  # 환경변수 OPENAI_API_KEY 사용

def clean_text(t: str) -> str:
    return re.sub(r"\s+", " ", str(t)).strip()

def too_similar(a: str, b: str) -> bool:
    """두 문장이 동일하거나 8자 이상 부분포함되면 유사."""
    a, b = a.strip(), b.strip()
    return a == b or (len(a) >= 8 and a in b) or (len(b) >= 8 and b in a)

def filter_by_similarity(candidates, references, keep_ratio=1.0):
    """후보 리스트에서 레퍼런스/서로 간 유사 문장 제거."""
    out = []
    for cand in candidates:
        c = clean_text(cand)
        if not c:
            continue
        # ref와의 유사성
        if any(too_similar(c, r) for r in references):
            continue
        # out(이미 선택된)과의 유사성
        if any(too_similar(c, o) for o in out):
            continue
        out.append(c)
    if keep_ratio < 1.0:
        k = max(1, int(len(out) * keep_ratio))
        out = random.sample(out, k=k)
    return out

def gpt_augment_one_batch(target_label, examples, n):
    shot = random.sample(examples, k=min(len(examples), 12))
    prompt = f"""역할: 당신은 한국어 감정 데이터 생성기입니다. 목표 라벨: "{target_label}"
아래는 {target_label} 라벨 실제 예시입니다:
{chr(10).join([f"{i+1}) '{s}'" for i,s in enumerate(shot)])}

요구사항:
- {target_label} 라벨 문장 {n}개 생성
- 길이 8~40자, 구어/문어/이모티콘 허용
- 예시와 중복/부분복사 금지
- JSON 배열만 출력
"""
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role":"user","content":prompt}],
        temperature=0.9,
        max_tokens=1600
    )
    # 응답은 JSON 배열 문자열이어야 함
    txt = resp.choices[0].message.content.strip()
    try:
        arr = json.loads(txt)
        if isinstance(arr, list):
            return [clean_text(x) for x in arr if isinstance(x,str)]
    except Exception:
        pass
    # 실패 시 라인별 분리해서라도 회수
    lines = [clean_text(x) for x in re.split(r"[\n\r]+", txt) if x.strip()]
    return lines

def gpt_augment_balanced(train_df, target_cap_per_label, per_call=50, max_calls=10):
    """train_df에서 라벨별 부족분을 GPT로 채움."""
    from collections import Counter
    counts = Counter(train_df["Emotion"])
    rows = []
    for label in LABELS:
        need = max(0, target_cap_per_label - counts[label])
        if need == 0:
            continue
        print(f"[{label}] need={need}")
        examples = train_df[train_df["Emotion"]==label]["Sentence"].tolist()
        # 여러 번 호출해서 need 채우기
        got = []
        calls = min(max_calls, math.ceil(need/per_call))
        for _ in range(calls):
            batch = gpt_augment_one_batch(label, examples, n=per_call)
            batch = filter_by_similarity(batch, examples + got)
            got.extend(batch)
            print(f"  +{len(batch)}  (cum={len(got)}/{need})")
            if len(got) >= need:
                break
        got = got[:need]
        rows += [(s, label) for s in got]
    if not rows:
        return train_df
    aug_df = pd.DataFrame(rows, columns=["Sentence","Emotion"])
    print("Augmented new rows:", len(aug_df))
    new_train = pd.concat([train_df, aug_df], ignore_index=True)
    return new_train.sample(frac=1.0, random_state=SEED).reset_index(drop=True)



In [ ]:
import pandas as pd
train_df = pd.read_excel("/content/drive/MyDrive/금융공모전/exports/train_balanced_v2.xlsx")

In [ ]:
train_df["Emotion"].value_counts()

,count
Emotion,
슬픔,12866
분노,12866
기쁨,11985
불안,10839
상처,10556
당황,10002
중립,7907


In [ ]:
# === 4) 증강 실행 (train 내 최다 클래스 개수에 맞추기) ===
LABELS = ["분노", "기쁨", "슬픔", "불안", "상처", "당황", "중립"]
import os, json, re, random, math

train_counts = train_df["Emotion"].value_counts()
target_cap = int(train_counts.max())  # train 내 최대치
print("Target cap per label (train):", target_cap)

# 필요 시만 증강
if any(train_counts[label] < target_cap for label in LABELS):
    train_df_bal = gpt_augment_balanced(train_df, target_cap_per_label=target_cap,
                                        per_call=200, max_calls=30)
else:
    train_df_bal = train_df.copy()

print("Train(after aug) dist:\n", train_df_bal["Emotion"].value_counts().sort_index())

Target cap per label (train): 12866
[기쁨] need=881
  +86  (cum=86/881)
  +85  (cum=171/881)
  +94  (cum=265/881)
  +87  (cum=352/881)
  +82  (cum=434/881)
[불안] need=2027
  +91  (cum=91/2027)
  +95  (cum=186/2027)
  +94  (cum=280/2027)
  +86  (cum=366/2027)
  +94  (cum=460/2027)
  +101  (cum=561/2027)
  +96  (cum=657/2027)
  +99  (cum=756/2027)
  +94  (cum=850/2027)
  +91  (cum=941/2027)
  +97  (cum=1038/2027)
[상처] need=2310
  +75  (cum=75/2310)
  +91  (cum=166/2310)
  +86  (cum=252/2310)
  +95  (cum=347/2310)
  +94  (cum=441/2310)
  +79  (cum=520/2310)
  +92  (cum=612/2310)
  +101  (cum=713/2310)
  +79  (cum=792/2310)
  +100  (cum=892/2310)
  +89  (cum=981/2310)
  +93  (cum=1074/2310)
[당황] need=2864
  +76  (cum=76/2864)
  +64  (cum=140/2864)
  +83  (cum=223/2864)
  +74  (cum=297/2864)
  +69  (cum=366/2864)
  +67  (cum=433/2864)
  +73  (cum=506/2864)
  +79  (cum=585/2864)
  +80  (cum=665/2864)
  +82  (cum=747/2864)
  +70  (cum=817/2864)
  +68  (cum=885/2864)
  +72  (cum=957/2864)
  +77  

In [ ]:
import os
from datetime import datetime

# 출력 폴더 만들기
OUT_DIR = "/content/drive/MyDrive/금융공모전/exports"
os.makedirs(OUT_DIR, exist_ok=True)


train_path = f"{OUT_DIR}/train_balanced_v3.xlsx"
# val_path   = f"{OUT_DIR}/val.xlsx"
# test_path  = f"{OUT_DIR}/test.xlsx"

# 저장
train_df_bal.to_excel(train_path, index=False)
print("Saved:")
print(train_path)


Saved:
/content/drive/MyDrive/금융공모전/exports/train_balanced_v3.xlsx


In [ ]:
train_df_bal["Emotion"].value_counts()

,count
Emotion,
분노,12866
슬픔,12866
기쁨,12419
불안,11877
상처,11630
당황,11115
중립,10752


# 학습

In [ ]:
!pip uninstall -y torch torchvision torchaudio
!pip install --index-url https://download.pytorch.org/whl/cpu torch==2.4.0


Found existing installation: torch 2.1.2+cu121
Uninstalling torch-2.1.2+cu121:
  Successfully uninstalled torch-2.1.2+cu121
Found existing installation: torchvision 0.16.2+cu121
Uninstalling torchvision-0.16.2+cu121:
  Successfully uninstalled torchvision-0.16.2+cu121
Found existing installation: torchaudio 2.1.2+cu121
Uninstalling torchaudio-2.1.2+cu121:
  Successfully uninstalled torchaudio-2.1.2+cu121
Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.1/195.1 MB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.19 requires torchvision>=0.11, which is not installed.
timm 1.0.19 requires torchvision, which is not installed.


In [ ]:
import torch
print(torch.__version__)          # 2.4.0+cpu
print(hasattr(torch, "uint64"))   # True 여야 함


2.4.0+cpu
True


In [ ]:
# === 1) 기본 준비 ===
import os, json, re, random, math
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_excel("/content/drive/MyDrive/금융공모전/exports/train_balanced.xlsx")

In [ ]:
train_df_bal = pd.read_excel("/content/drive/MyDrive/금융공모전/exports/train_balanced_v3.xlsx")
val_df = pd.read_excel("/content/drive/MyDrive/금융공모전/exports/val.xlsx")
test_df = pd.read_excel("/content/drive/MyDrive/금융공모전/exports/test.xlsx")

In [ ]:
train_df_bal["Emotion"].value_counts()

,count
Emotion,
분노,12866
슬픔,12866
기쁨,12419
불안,11877
상처,11630
당황,11115
중립,10752


In [ ]:
val_df["Emotion"].value_counts()

,count
Emotion,
분노,1608
슬픔,1539
기쁨,1337
불안,1043
상처,1015
당황,981
중립,484


In [ ]:
train["Emotion"].value_counts()

,count
Emotion,
분노,12866
슬픔,12823
기쁨,11208
불안,8846
상처,8631
당황,8308
중립,4380


In [ ]:
# === 5) HF Datasets 형태로 변환 ===
from datasets import Dataset, DatasetDict
LABELS = ["분노", "기쁨", "슬픔", "불안", "상처", "당황", "중립"]

# 라벨 인코딩
label2id = {l:i for i,l in enumerate(LABELS)}
id2label = {i:l for l,i in label2id.items()}

def to_hf(ds: pd.DataFrame):
    return Dataset.from_pandas(
        ds[["Sentence","Emotion"]].rename(columns={"Sentence":"text","Emotion":"label_str"}),
        preserve_index=False
    ).map(lambda x: {"label": label2id[x["label_str"]]})

raw_dset = DatasetDict({
    "train": to_hf(train_df_bal),
    "validation": to_hf(val_df),
    "test": to_hf(test_df),
})

# === 6) 토크나이저/모델 로드 ===
from transformers import AutoTokenizer, LlamaForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding

MODEL_NAME = "openlm-research/open_llama_3b"   # <- 여기를 원하는 Llama 계열로 교체
MAX_LEN = 128

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
# pad 토큰이 없을 수 있어 대비
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tok_fn(batch):
    out = tokenizer(batch["text"], padding=False, truncation=True, max_length=MAX_LEN)
    out["labels"] = batch["label"]
    return out

tokenized = raw_dset.map(tok_fn, batched=True, remove_columns=["text","label_str","label"])
collator = DataCollatorWithPadding(tokenizer=tokenizer, pad_to_multiple_of=8)

model = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(LABELS),
    id2label=id2label,
    label2id=label2id
)

# === 7) 메트릭 (정확도/매크로 F1) ===
import evaluate
acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    import numpy as np
    preds = logits.argmax(axis=-1)
    return {
        "accuracy": acc_metric.compute(predictions=preds, references=labels)["accuracy"],
        "f1_macro": f1_metric.compute(predictions=preds, references=labels, average="macro")["f1"],
    }

# === 8) 학습 설정 + EarlyStopping ===
from transformers import EarlyStoppingCallback

OUTPUT_DIR = "/content/llama-emotion-clf"
bsz = 8    # GPU 여유에 맞춰 조절
grad_acc = 2
lr = 2e-5
epochs = 3

args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    learning_rate=lr,
    per_device_train_batch_size=bsz,
    per_device_eval_batch_size=bsz,
    gradient_accumulation_steps=grad_acc,
    num_train_epochs=epochs,
    evaluation_strategy="steps",
    eval_steps=200,
    logging_steps=100,
    save_total_limit=2,
    save_strategy="steps",
    save_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=False, # gpu에선 True
    seed=SEED,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

train_result = trainer.train()
print("Best checkpoint:", trainer.state.best_model_checkpoint)

# === 9) 최종 성능: train/val 분리 출력 ===
print("\n[Evaluate: TRAIN]")
print(trainer.evaluate(tokenized["train"]))
print("\n[Evaluate: VAL]")
print(trainer.evaluate(tokenized["validation"]))

# === 10) Test 세트 평가 ===
print("\n[Evaluate: TEST]")
test_metrics = trainer.evaluate(tokenized["test"])
print(test_metrics)

# === 11) 저장 ===
trainer.save_model(os.path.join(OUTPUT_DIR, "final"))
tokenizer.save_pretrained(os.path.join(OUTPUT_DIR, "final"))


Map:   0%|          | 0/67062 [00:00<?, ? examples/s]

Map:   0%|          | 0/8007 [00:00<?, ? examples/s]

Map:   0%|          | 0/8008 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/593 [00:00<?, ?B/s]

...

Map:   0%|          | 0/67062 [00:00<?, ? examples/s]

Map:   0%|          | 0/8007 [00:00<?, ? examples/s]

Map:   0%|          | 0/8008 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.85G [00:00<?, ?B/s]